## Preparação

In [14]:
import json
from pathlib import Path
from dataclasses import dataclass
from typing import Optional
from datetime import date

print("Bibliotecas carregadas!")

Bibliotecas carregadas!


## Passo 1: Problema Atual

Vamos ver como estamos trabalhando agora:

In [15]:
# Carregar um filme
with open("dados/filme_550.json") as f:
    filme = json.load(f)

print("Tipo do filme:", type(filme))
print()
print("Acessando dados:")
print(f"  Título: {filme['title']}")
print(f"  Ano: {filme['release_date'][:4]}")

Tipo do filme: <class 'dict'>

Acessando dados:
  Título: Clube da Luta
  Ano: 1999


### Problemas desta abordagem:

1. **Não sabemos quais campos existem**
   - Temos que lembrar os nomes
   - Não há autocomplete

2. **Sem validação**
   - Podemos colocar qualquer valor
   - Erros só aparecem quando executamos

3. **Código repetitivo**
   - Sempre acessamos com `filme['campo']`
   - Se mudar o nome, temos que mudar em todo lugar

### Exemplo de problema:

In [16]:
# Tentativa com erro de digitação
try:
    print(filme['tittle'])  # Erro: 'tittle' em vez de 'title'
except KeyError as e:
    print(f"Erro: Campo {e} não existe!")
    print("Este erro só aparece quando executamos.")

Erro: Campo 'tittle' não existe!
Este erro só aparece quando executamos.


## Passo 2: O que são Classes?

Classes são **moldes** para criar objetos.

### Analogia: Receita de Bolo

```
Receita (classe):
  - 3 ovos
  - 2 xícaras de farinha
  - 1 xícara de açúcar
  Modo de preparo...

Bolo 1 (objeto): feito com a receita
Bolo 2 (objeto): feito com a receita
Bolo 3 (objeto): feito com a receita
```

- **Classe**: A receita (estrutura)
- **Objeto**: O bolo real (instância)

### Classe simples:

In [17]:
class Pessoa:
    def __init__(self, nome: str, idade: int):
        self.nome = nome
        self.idade = idade
    
    def apresentar(self):
        return f"Olá, meu nome é {self.nome} e tenho {self.idade} anos"

# Criar objetos
pessoa1 = Pessoa("Ana", 25)
pessoa2 = Pessoa("João", 30)

print(pessoa1.apresentar())
print(pessoa2.apresentar())

Olá, meu nome é Ana e tenho 25 anos
Olá, meu nome é João e tenho 30 anos


## Passo 3: Dataclasses

Python tem uma forma mais simples de criar classes: **dataclasses**.

### Mesmo exemplo com dataclass:

In [18]:
from dataclasses import dataclass

@dataclass
class PessoaDataclass:
    nome: str
    idade: int
    
    def apresentar(self):
        return f"Olá, meu nome é {self.nome} e tenho {self.idade} anos"

# Criar objeto
pessoa = PessoaDataclass("Maria", 28)

print(pessoa.apresentar())
print()
print("Representação automática:")
print(pessoa)

Olá, meu nome é Maria e tenho 28 anos

Representação automática:
PessoaDataclass(nome='Maria', idade=28)


### Vantagens de dataclasses:

1. Menos código (não precisa de `__init__`)
2. Representação automática (`__repr__`)
3. Comparação automática (`__eq__`)
4. Tipos claramente definidos

## Passo 4: Criar Classe para Filme

Vamos criar uma classe para representar filmes.

### Versão 1: Campos básicos

In [19]:
@dataclass
class Movie:
    """Representa um filme do TMDb."""
    id: int
    title: str
    original_title: str
    release_date: str
    vote_average: float
    vote_count: int

print("Classe Movie criada!")

Classe Movie criada!


### Testar a classe:

In [20]:
# Criar um filme
filme_obj = Movie(
    id=550,
    title="Clube da Luta",
    original_title="Fight Club",
    release_date="1999-10-15",
    vote_average=8.4,
    vote_count=27000
)

print(filme_obj)
print()
print(f"Título: {filme_obj.title}")
print(f"Nota: {filme_obj.vote_average}/10")

Movie(id=550, title='Clube da Luta', original_title='Fight Club', release_date='1999-10-15', vote_average=8.4, vote_count=27000)

Título: Clube da Luta
Nota: 8.4/10


### Benefícios:

1. **Autocomplete funciona**
   - Quando você digita `filme_obj.` o editor sugere os campos

2. **Tipos são verificados**
   - O editor avisa se você passar tipo errado

3. **Código mais legível**
   - `filme.title` é mais claro que `filme['title']`

## Passo 5: Adicionar Mais Campos

Vamos adicionar mais informações úteis:

In [21]:
@dataclass
class Movie:
    """Representa um filme do TMDb."""
    id: int
    title: str
    original_title: str
    release_date: str
    vote_average: float
    vote_count: int
    overview: str
    popularity: float
    adult: bool
    original_language: str
    
    # Campos opcionais (podem não existir)
    budget: Optional[int] = None
    revenue: Optional[int] = None
    runtime: Optional[int] = None

print("Classe Movie expandida criada!")

Classe Movie expandida criada!


### O que é Optional?

`Optional[int]` significa:
- Pode ser um `int`
- Ou pode ser `None`

Usamos para campos que nem sempre existem.

## Passo 6: Converter JSON para Objeto

Vamos criar um método para transformar JSON em objeto Movie.

In [22]:
@dataclass
class Movie:
    """Representa um filme do TMDb."""
    id: int
    title: str
    original_title: str
    release_date: str
    vote_average: float
    vote_count: int
    overview: str
    popularity: float
    adult: bool
    original_language: str
    budget: Optional[int] = None
    revenue: Optional[int] = None
    runtime: Optional[int] = None
    
    @classmethod
    def from_json(cls, data: dict) -> 'Movie':
        """Cria um Movie a partir de um dicionário JSON."""
        return cls(
            id=data['id'],
            title=data['title'],
            original_title=data['original_title'],
            release_date=data['release_date'],
            vote_average=data['vote_average'],
            vote_count=data['vote_count'],
            overview=data.get('overview', ''),
            popularity=data['popularity'],
            adult=data['adult'],
            original_language=data['original_language'],
            budget=data.get('budget'),
            revenue=data.get('revenue'),
            runtime=data.get('runtime')
        )

print("Classe Movie com from_json criada!")

Classe Movie com from_json criada!


### Testar conversão:

In [23]:
# Carregar JSON
with open("dados/filme_550.json") as f:
    data = json.load(f)

# Converter para objeto
filme = Movie.from_json(data)

print("Filme convertido:")
print(f"  ID: {filme.id}")
print(f"  Título: {filme.title}")
print(f"  Ano: {filme.release_date[:4]}")
print(f"  Nota: {filme.vote_average}/10")
print(f"  Orçamento: ${filme.budget:,}" if filme.budget else "  Orçamento: N/A")
print(f"  Duração: {filme.runtime} min" if filme.runtime else "  Duração: N/A")

Filme convertido:
  ID: 550
  Título: Clube da Luta
  Ano: 1999
  Nota: 8.4/10
  Orçamento: $63,000,000
  Duração: 139 min


## Passo 7: Adicionar Métodos Úteis

Vamos adicionar métodos para facilitar o uso da classe.

In [24]:
@dataclass
class Movie:
    """Representa um filme do TMDb."""
    id: int
    title: str
    original_title: str
    release_date: str
    vote_average: float
    vote_count: int
    overview: str
    popularity: float
    adult: bool
    original_language: str
    budget: Optional[int] = None
    revenue: Optional[int] = None
    runtime: Optional[int] = None
    
    @classmethod
    def from_json(cls, data: dict) -> 'Movie':
        """Cria um Movie a partir de um dicionário JSON."""
        return cls(
            id=data['id'],
            title=data['title'],
            original_title=data['original_title'],
            release_date=data['release_date'],
            vote_average=data['vote_average'],
            vote_count=data['vote_count'],
            overview=data.get('overview', ''),
            popularity=data['popularity'],
            adult=data['adult'],
            original_language=data['original_language'],
            budget=data.get('budget'),
            revenue=data.get('revenue'),
            runtime=data.get('runtime')
        )
    
    def get_year(self) -> Optional[int]:
        """Extrai o ano de lançamento."""
        if self.release_date:
            return int(self.release_date[:4])
        return None
    
    def is_profitable(self) -> bool:
        """Verifica se o filme foi lucrativo."""
        if self.budget and self.revenue:
            return self.revenue > self.budget
        return False
    
    def get_profit(self) -> Optional[int]:
        """Calcula o lucro do filme."""
        if self.budget and self.revenue:
            return self.revenue - self.budget
        return None
    
    def is_well_rated(self, threshold: float = 7.0) -> bool:
        """Verifica se o filme é bem avaliado."""
        return self.vote_average >= threshold

print("Classe Movie completa criada!")

Classe Movie completa criada!


### Testar métodos:

In [25]:
# Criar objeto
with open("dados/filme_550.json") as f:
    filme = Movie.from_json(json.load(f))

print(f"Filme: {filme.title}")
print()
print(f"Ano: {filme.get_year()}")
print(f"Bem avaliado? {filme.is_well_rated()}")

if filme.budget and filme.revenue:
    print(f"Lucrativo? {filme.is_profitable()}")
    profit = filme.get_profit()
    if profit:
        print(f"Lucro: ${profit:,}")

Filme: Clube da Luta

Ano: 1999
Bem avaliado? True
Lucrativo? True
Lucro: $37,853,753


## Passo 8: Adicionar Validações

Vamos garantir que os dados estão corretos.

In [26]:
@dataclass
class Movie:
    """Representa um filme do TMDb com validações."""
    id: int
    title: str
    original_title: str
    release_date: str
    vote_average: float
    vote_count: int
    overview: str
    popularity: float
    adult: bool
    original_language: str
    budget: Optional[int] = None
    revenue: Optional[int] = None
    runtime: Optional[int] = None
    
    def __post_init__(self):
        """Validações após criação do objeto."""
        # Validar ID
        if self.id <= 0:
            raise ValueError(f"ID deve ser positivo: {self.id}")
        
        # Validar título
        if not self.title or not self.title.strip():
            raise ValueError("Título não pode ser vazio")
        
        # Validar nota
        if not 0 <= self.vote_average <= 10:
            raise ValueError(f"Nota deve estar entre 0 e 10: {self.vote_average}")
        
        # Validar contagem de votos
        if self.vote_count < 0:
            raise ValueError(f"Contagem de votos não pode ser negativa: {self.vote_count}")
    
    @classmethod
    def from_json(cls, data: dict) -> 'Movie':
        """Cria um Movie a partir de um dicionário JSON."""
        return cls(
            id=data['id'],
            title=data['title'],
            original_title=data['original_title'],
            release_date=data['release_date'],
            vote_average=data['vote_average'],
            vote_count=data['vote_count'],
            overview=data.get('overview', ''),
            popularity=data['popularity'],
            adult=data['adult'],
            original_language=data['original_language'],
            budget=data.get('budget'),
            revenue=data.get('revenue'),
            runtime=data.get('runtime')
        )
    
    def get_year(self) -> Optional[int]:
        """Extrai o ano de lançamento."""
        if self.release_date:
            return int(self.release_date[:4])
        return None
    
    def is_profitable(self) -> bool:
        """Verifica se o filme foi lucrativo."""
        if self.budget and self.revenue:
            return self.revenue > self.budget
        return False
    
    def get_profit(self) -> Optional[int]:
        """Calcula o lucro do filme."""
        if self.budget and self.revenue:
            return self.revenue - self.budget
        return None
    
    def is_well_rated(self, threshold: float = 7.0) -> bool:
        """Verifica se o filme é bem avaliado."""
        return self.vote_average >= threshold

print("Classe Movie com validações criada!")

Classe Movie com validações criada!


### Testar validações:

In [27]:
# Teste 1: Dados válidos
print("Teste 1: Dados válidos")
try:
    filme_ok = Movie(
        id=1,
        title="Teste",
        original_title="Test",
        release_date="2020-01-01",
        vote_average=7.5,
        vote_count=100,
        overview="Filme de teste",
        popularity=10.0,
        adult=False,
        original_language="en"
    )
    print("  ✓ Criado com sucesso")
except ValueError as e:
    print(f"  ✗ Erro: {e}")

print()

# Teste 2: ID inválido
print("Teste 2: ID inválido (negativo)")
try:
    filme_erro = Movie(
        id=-1,
        title="Teste",
        original_title="Test",
        release_date="2020-01-01",
        vote_average=7.5,
        vote_count=100,
        overview="Filme de teste",
        popularity=10.0,
        adult=False,
        original_language="en"
    )
    print("  ✗ Deveria ter dado erro!")
except ValueError as e:
    print(f"  ✓ Erro capturado: {e}")

print()

# Teste 3: Nota inválida
print("Teste 3: Nota inválida (> 10)")
try:
    filme_erro = Movie(
        id=1,
        title="Teste",
        original_title="Test",
        release_date="2020-01-01",
        vote_average=15.0,  # Inválido!
        vote_count=100,
        overview="Filme de teste",
        popularity=10.0,
        adult=False,
        original_language="en"
    )
    print("  ✗ Deveria ter dado erro!")
except ValueError as e:
    print(f"  ✓ Erro capturado: {e}")

Teste 1: Dados válidos
  ✓ Criado com sucesso

Teste 2: ID inválido (negativo)
  ✓ Erro capturado: ID deve ser positivo: -1

Teste 3: Nota inválida (> 10)
  ✓ Erro capturado: Nota deve estar entre 0 e 10: 15.0


## Passo 9: Processar Todos os Filmes

Vamos carregar todos os filmes como objetos.

In [28]:
# Listar arquivos de filmes
dados_path = Path("dados")
arquivos = list(dados_path.glob("filme_*.json"))

print(f"Encontrados {len(arquivos)} arquivos")

Encontrados 11 arquivos


In [29]:
# Carregar todos os filmes
filmes = []
erros = []

for arquivo in arquivos:
    try:
        with open(arquivo) as f:
            data = json.load(f)
        
        filme = Movie.from_json(data)
        filmes.append(filme)
    
    except Exception as e:
        erros.append((arquivo.name, str(e)))

print(f"Carregados: {len(filmes)} filmes")
print(f"Erros: {len(erros)}")

if erros:
    print("\nErros encontrados:")
    for arquivo, erro in erros:
        print(f"  {arquivo}: {erro}")

Carregados: 11 filmes
Erros: 0


## Passo 10: Análises com Objetos

Agora fica muito mais fácil analisar os dados!

### Filmes bem avaliados:

In [30]:
bem_avaliados = [f for f in filmes if f.is_well_rated(8.0)]

print(f"Filmes com nota >= 8.0: {len(bem_avaliados)}")
print()

for filme in bem_avaliados[:5]:
    print(f"  {filme.title:40} {filme.vote_average}/10")

Filmes com nota >= 8.0: 1

  Clube da Luta                            8.4/10


### Filmes por década:

In [31]:
from collections import Counter

# Agrupar por década
decadas = []
for filme in filmes:
    ano = filme.get_year()
    if ano:
        decada = (ano // 10) * 10
        decadas.append(decada)

contagem = Counter(decadas)

print("Filmes por década:")
for decada in sorted(contagem.keys()):
    print(f"  {decada}s: {contagem[decada]} filmes")

Filmes por década:
  1980s: 1 filmes
  1990s: 1 filmes
  2020s: 9 filmes


### Filmes lucrativos:

In [32]:
lucrativos = [f for f in filmes if f.is_profitable()]

print(f"Filmes lucrativos: {len(lucrativos)}")
print()

# Ordenar por lucro
lucrativos_ordenados = sorted(
    lucrativos,
    key=lambda f: f.get_profit() or 0,
    reverse=True
)

print("Top 3 mais lucrativos:")
for i, filme in enumerate(lucrativos_ordenados[:3], 1):
    lucro = filme.get_profit()
    if lucro:
        print(f"{i}. {filme.title:40} ${lucro:,}")

Filmes lucrativos: 5

Top 3 mais lucrativos:
1. Zootopia 2                               $1,464,766,128
2. Avatar: Fogo e Cinzas                    $886,161,168
3. A Empregada                              $161,972,064


## Passo 11: Salvar Classe em Arquivo

Vamos salvar nossa classe para reutilizar depois.

In [33]:
# Criar código da classe
codigo_classe = '''"""Modelos de dados para o pipeline de filmes."""

from dataclasses import dataclass
from typing import Optional


@dataclass
class Movie:
    """Representa um filme do TMDb com validações."""
    id: int
    title: str
    original_title: str
    release_date: str
    vote_average: float
    vote_count: int
    overview: str
    popularity: float
    adult: bool
    original_language: str
    budget: Optional[int] = None
    revenue: Optional[int] = None
    runtime: Optional[int] = None
    
    def __post_init__(self):
        """Validações após criação do objeto."""
        if self.id <= 0:
            raise ValueError(f"ID deve ser positivo: {self.id}")
        
        if not self.title or not self.title.strip():
            raise ValueError("Título não pode ser vazio")
        
        if not 0 <= self.vote_average <= 10:
            raise ValueError(f"Nota deve estar entre 0 e 10: {self.vote_average}")
        
        if self.vote_count < 0:
            raise ValueError(f"Contagem de votos não pode ser negativa: {self.vote_count}")
    
    @classmethod
    def from_json(cls, data: dict) -> 'Movie':
        """Cria um Movie a partir de um dicionário JSON."""
        return cls(
            id=data['id'],
            title=data['title'],
            original_title=data['original_title'],
            release_date=data['release_date'],
            vote_average=data['vote_average'],
            vote_count=data['vote_count'],
            overview=data.get('overview', ''),
            popularity=data['popularity'],
            adult=data['adult'],
            original_language=data['original_language'],
            budget=data.get('budget'),
            revenue=data.get('revenue'),
            runtime=data.get('runtime')
        )
    
    def get_year(self) -> Optional[int]:
        """Extrai o ano de lançamento."""
        if self.release_date:
            return int(self.release_date[:4])
        return None
    
    def is_profitable(self) -> bool:
        """Verifica se o filme foi lucrativo."""
        if self.budget and self.revenue:
            return self.revenue > self.budget
        return False
    
    def get_profit(self) -> Optional[int]:
        """Calcula o lucro do filme."""
        if self.budget and self.revenue:
            return self.revenue - self.budget
        return None
    
    def is_well_rated(self, threshold: float = 7.0) -> bool:
        """Verifica se o filme é bem avaliado."""
        return self.vote_average >= threshold
'''

# Salvar arquivo
with open("models.py", "w", encoding="utf-8") as f:
    f.write(codigo_classe)

print("Classe salva em models.py")

Classe salva em models.py


### Testar importação:

In [34]:
# Importar do arquivo
from models import Movie as MovieImportado

# Testar
with open("dados/filme_550.json") as f:
    filme_teste = MovieImportado.from_json(json.load(f))

print(f"Importação funcionou!")
print(f"Filme: {filme_teste.title}")

Importação funcionou!
Filme: Clube da Luta


## Resumo do Bloco 3

### O que fizemos:

1. Entendemos os problemas de trabalhar apenas com dicionários
2. Aprendemos o conceito de classes
3. Usamos dataclasses para simplificar
4. Criamos a classe Movie
5. Adicionamos validações
6. Criamos métodos úteis
7. Convertemos todos os JSONs em objetos
8. Fizemos análises mais fáceis
9. Salvamos a classe em arquivo reutilizável

### O que aprendemos:

- O que são classes e objetos
- Como usar dataclasses
- Como adicionar validações
- Como criar métodos úteis
- Vantagens de programação orientada a objetos
- Como organizar código em módulos

### Arquivos criados:

```
models.py    - Classe Movie reutilizável
```

## Verificação Final

In [35]:
print("Verificação Final")
print("=" * 50)

# Verificar arquivo models.py
if Path("models.py").exists():
    print("models.py: OK")
else:
    print("models.py: NÃO encontrado")

# Verificar se consegue importar
try:
    from models import Movie
    print("Importação: OK")
except ImportError:
    print("Importação: ERRO")

# Verificar se processou filmes
print(f"Filmes carregados: {len(filmes)}")

print("=" * 50)
print()
print("Se tudo está OK, você está pronto para o próximo bloco!")

Verificação Final
models.py: OK
Importação: OK
Filmes carregados: 11

Se tudo está OK, você está pronto para o próximo bloco!


## Próximo Passo

Agora que temos nossos dados bem estruturados em classes Python, vamos para o **Bloco 4**:

**04_versionamento.ipynb** - Vamos aprender a versionar nosso código com Git para nunca perder trabalho.

### Pausa

Faça uma pausa de 5-10 minutos antes de continuar.
- Tome água
- Levante e estique
- Revise o conceito de classes